# Chinese Translation

In [1]:
!pip install --upgrade "numpy>=1.20.3"


In [4]:
import openai
import pandas as pd
import re

# Load your dataset from an Excel file
col_names_comment = ['key',"video_link","cid","comment_time","likes","replies","comment_IP","comment",'sentiment']
col_names_video=["video_id","video_description","video_topic","video_length","video_link","release_date","release_account","likes","num_comments","archives","shares"]

li_comments = pd.read_excel("data/annotated_li_comment.xlsx")
li_video = pd.read_excel("data/cleaned_li_video.xlsx")
aito_comments = pd.read_excel("data/annotated_aito_comment.xlsx")
aito_video = pd.read_excel("data/cleaned_aito_video.xlsx")
li_comments.columns = col_names_comment
aito_comments.columns = col_names_comment
li_video.columns = col_names_video
aito_video.columns = col_names_video




In [5]:
li_comments

,key,video_link,cid,comment_time,likes,replies,comment_IP,comment,sentiment
0,0,https://www.douyin.com/video/6724238509927976206,6724315314135789568,2019-08-13 00:17:49,129,0,无,渐强扶弱，拜强欺弱，根本区别再大的博物馆都没用,negative
1,1,https://www.douyin.com/video/6724238509927976206,6724307710353139712,2019-08-12 23:48:15,274,2,无,英国留学让你更坚定认为自己是英国人,neutral
2,2,https://www.douyin.com/video/6724238509927976206,6726852199668480000,2019-08-19 20:22:10,107,0,无,老爷子也是一个伟人,positive
3,3,https://www.douyin.com/video/6724238509927976206,6724309456349589504,2019-08-12 23:55:03,214,2,无,李光耀 要有认祖的初心,neutral
4,4,https://www.douyin.com/video/6724238509927976206,6724324047796400128,2019-08-13 00:51:43,535,3,无,看不明白吧，这回家多少是华人，确不是中国的铁盟。值得研究。,neutral
...,...,...,...,...,...,...,...,...,...
42335,42335,https://www.douyin.com/video/7350480407307078927,7350522030523710464,2024-03-26 12:19:41,2,0,湖北,理想的,neutral
42336,42336,https://www.douyin.com/video/7350480407307078927,7350527617931410432,2024-03-26 12:41:19,2,0,河南,漂亮的豪车,positive
42337,42337,https://www.douyin.com/video/7350480407307078927,7350521168641030144,2024-03-26 12:16:20,6,0,湖南,现在的车越来越理想化了，科技含量高,positive
42338,42338,https://www.douyin.com/video/7350480407307078927,7350523010317200384,2024-03-26 12:23:27,2,0,福建,霸气,positive


In [8]:
li_video

,video_id,video_description,video_topic,video_length,video_link,release_date,release_account,likes,num_comments,archives,shares
0,6953522111382060032,智能、科技、安全、续航都哇塞，除了理想ONE还有谁！,创造移动的家,00:01:30,https://www.douyin.com/video/6953522111382064397,2021-04-21 20:30:04,南哥说车,601017,5299,3031,2304
1,6983591010307787776,理想ONE水银事件,星巢计划 星巢评车人,00:01:19,https://www.douyin.com/video/6983591010307788062,2021-07-11 17:00:03,毒辣车评,556388,9473,1900,4683
2,6953924024518839296,退货三部曲 ！姜还是老的辣，俺丈人真厉害,农村搞笑段子 搞笑视频,00:02:15,https://www.douyin.com/video/6953924024518839593,2021-04-22 18:16:51,理想•村里人,474582,4943,6898,16993
3,7261621428779338752,你的理想同学响应速度怎么样？该说不说理想OTA更新的是真勤快，各位车主有没有好用的分享码推荐一下！,理想汽车 理想l7 理想l8 理想l9,00:01:53,https://www.douyin.com/video/7261621428779339064,2023-07-30 22:40:01,理想兽,415949,83659,17916,110598
4,6979613318105353216,建党百年重磅微视频：《理想》。曾经他们义无反顾，今天我们当仁不让！,NaN,00:02:23,https://www.douyin.com/video/6979613318105353479,2021-06-30 23:45:38,人民日报,374689,7792,7634,43308
...,...,...,...,...,...,...,...,...,...,...,...
78,7154979963052067840,别争，理想L8就是近乎完美的。,比亚迪 理想,00:01:51,https://www.douyin.com/video/7154979963052068126,2022-10-16 13:37:03,爆哥,2325,867,105,362
79,6864048392897694720,没有像样的理想，早晚会看出差距。让他们自我验证不低等，跟要了他们的命似的。,星座,00:00:59,https://www.douyin.com/video/6864048392897694983,2020-08-23 13:33:29,GoddessCheng程女神,2272,285,315,212
80,7229625612716100608,理想的销量还会继续涨下去吗？,理想l7 理想l8 理想l9 理想汽车,00:01:08,https://www.douyin.com/video/7229625612716100897,2023-05-05 17:19:56,伞电有理想,2062,2126,147,417
81,7319715401435220992,好多理想L9都会升级刹车卡钳,河南理想改装升级 郑州理想改装升级 理想L9改装升级 理想刹车卡钳改装升级 汽车改装升级,00:03:31,https://www.douyin.com/video/7319715401435221274,2024-01-03 18:00:00,森之润理想哥,2051,2492,432,365


In [10]:
irrelavant_links= [
    "https://www.douyin.com/video/6953924024518839593",
    "https://www.douyin.com/video/6979613318105353479",
    "https://www.douyin.com/video/6724238509927976206",
    "https://www.douyin.com/video/7127604715617127684",
    "https://www.douyin.com/video/7096756034928119081",
    "https://www.douyin.com/video/7027705719998713129",
    "https://www.douyin.com/video/7092745278658268454",
    "https://www.douyin.com/video/6804754290553457933",
    "https://www.douyin.com/video/7093853232191048997",
    "https://www.douyin.com/video/7104836122911591710",
    "https://www.douyin.com/video/6864048392897694983"
]

li_video = li_video[~li_video["video_link"].isin(irrelavant_links)]
links = li_video["video_link"].unique()
li_comments = li_comments[li_comments["video_link"].isin(links)]

In [11]:
li_video

,video_id,video_description,video_topic,video_length,video_link,release_date,release_account,likes,num_comments,archives,shares
0,6953522111382060032,智能、科技、安全、续航都哇塞，除了理想ONE还有谁！,创造移动的家,00:01:30,https://www.douyin.com/video/6953522111382064397,2021-04-21 20:30:04,南哥说车,601017,5299,3031,2304
1,6983591010307787776,理想ONE水银事件,星巢计划 星巢评车人,00:01:19,https://www.douyin.com/video/6983591010307788062,2021-07-11 17:00:03,毒辣车评,556388,9473,1900,4683
3,7261621428779338752,你的理想同学响应速度怎么样？该说不说理想OTA更新的是真勤快，各位车主有没有好用的分享码推荐一下！,理想汽车 理想l7 理想l8 理想l9,00:01:53,https://www.douyin.com/video/7261621428779339064,2023-07-30 22:40:01,理想兽,415949,83659,17916,110598
5,7248878777713888256,据说理想出了一个上下班能自己开的功能。,创造移动的家 理想l7,00:04:25,https://www.douyin.com/video/7248878777713888568,2023-06-26 17:00:44,小刚学长,288472,34462,15882,108380
6,7111549767959792640,这理想L9的第二排，就是一个全家桶！,50万买理想L9值不值,00:01:41,https://www.douyin.com/video/7111549767959792909,2022-06-21 21:01:00,陈震同学,275772,48469,16260,87310
...,...,...,...,...,...,...,...,...,...,...,...
75,7234878441038941184,理想L8 预算三万！怎么改？ 就这样改！,理想l9 理想l7 理想l8 理想汽车 理想l8升级,00:02:08,https://www.douyin.com/video/7234878441038941500,2023-05-19 21:03:31,可可的理想,4455,354,1181,2651
78,7154979963052067840,别争，理想L8就是近乎完美的。,比亚迪 理想,00:01:51,https://www.douyin.com/video/7154979963052068126,2022-10-16 13:37:03,爆哥,2325,867,105,362
80,7229625612716100608,理想的销量还会继续涨下去吗？,理想l7 理想l8 理想l9 理想汽车,00:01:08,https://www.douyin.com/video/7229625612716100897,2023-05-05 17:19:56,伞电有理想,2062,2126,147,417
81,7319715401435220992,好多理想L9都会升级刹车卡钳,河南理想改装升级 郑州理想改装升级 理想L9改装升级 理想刹车卡钳改装升级 汽车改装升级,00:03:31,https://www.douyin.com/video/7319715401435221274,2024-01-03 18:00:00,森之润理想哥,2051,2492,432,365


In [12]:

# Check for missing values in each column
missing_values_per_column = li_comments.isnull().any()

print("Columns with missing values:")
print(missing_values_per_column)


Columns with missing values:
key             False
video_link      False
cid             False
comment_time    False
likes           False
replies         False
comment_IP      False
comment         False
sentiment        True
dtype: bool


In [13]:

# Check for missing values in each column
missing_values_per_column = aito_comments.isnull().any()

print("Columns with missing values:")
print(missing_values_per_column)

Columns with missing values:
key             False
video_link      False
cid             False
comment_time    False
likes           False
replies         False
comment_IP      False
comment         False
sentiment        True
dtype: bool


English translation

In [17]:

# Assuming you've already set your API key
openai.api_key = ''

def translate_comment(content):
    system_prompt = "You are a highly knowledgeable assistant asked to translate Chinese into English"
    user_prompt = (
        f"Translate:\n{content}\n\n"
        "Your output must only be the translated content."
    )
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0125",  # Adjust model as necessary
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
        )
        translations = response.choices[0].message['content']
        #print(translations)
        #parts = annotations.split("\n\n")  # Assuming each section is separated by two newlines
        return translations  # Return the first four sections
    except Exception as e:
        print(f"An error occurred: {e}")
        return ""



translate_comment("李想在我眼里就是个“司机”")

'In my eyes, Li is just a "driver."'

# EYEBALL

In [28]:
num_processed = 0 # Store the number of comments annoted
translations = [] # Remeber just append   don't run it second time

In [29]:
import random

 
# sample 5 comment every batch (batch = 100 here)
def process_comments_in_batches(df,num_processed, batch_size=10, sample_size=5):
    if (len(df)-num_processed)<100:
        batch_size = len(df)-num_processed
        num_processed = len(df)
    results = []
    translation = []
    try:
        comments = df.iloc[num_processed:num_processed+batch_size]['comment']
    except:
        comments = df.iloc[num_processed:]['comment']
    
    for comment in comments:
        trans = translate_comment(comment)
        result = (comment,str(trans))
        translation.append(trans)
        results.append(result)
        
    num_processed += batch_size
    sampled_comments = random.sample(results, min(sample_size, 5)) 
    return results,sampled_comments,translation,num_processed



In [30]:
results, sampled_comments,translation_1,num_processed_1= process_comments_in_batches(li_comments,num_processed)

In [31]:
results

[('我感觉你什么实话都敢说', 'I feel like you dare to speak the truth about everything.'),
 ('你不能拿好几辆车最强的点，去对比理想ONE，这不公平吧？',
  "You can't take the best points of several cars to compare with the ONE ideal, isn't that unfair?"),
 ('理想one 单枪匹马pk其他所有车型',
  'Translation:\nThe ideal one goes solo to compete with all other car models'),
 ('你太狠了', 'You are too harsh.'),
 ('区长好“突出”啊！', 'Hello District Mayor, "stand out" ah!'),
 ('你就好比理想one！嘴碎就是你最强的一环，你搞得这些人买那个车都纠结，每个车都有优点！你和其他车评人区别就是找出每个车的缺点播流量！像你说的显得你很先锋，你也无敌',
  'You are like the ideal one! Being talkative is your strongest suit, making these people hesitate about which car to buy, every car has its advantages! The difference between you and other car reviewers is that you focus on finding the shortcomings of each car to attract attention! It seems like you are very avant-garde, you are also invincible.'),
 ('谁都比不过我太难了', "No one can compare to me, it's too difficult."),
 ('想问问200万左右哪款车比较好，最近想着买个保险，怕自己单车碰到',
  'I want to ask which car around 2 million wo

In [32]:
translation_1

['I feel like you dare to speak the truth about everything.',
 "You can't take the best points of several cars to compare with the ONE ideal, isn't that unfair?",
 'Translation:\nThe ideal one goes solo to compete with all other car models',
 'You are too harsh.',
 'Hello District Mayor, "stand out" ah!',
 'You are like the ideal one! Being talkative is your strongest suit, making these people hesitate about which car to buy, every car has its advantages! The difference between you and other car reviewers is that you focus on finding the shortcomings of each car to attract attention! It seems like you are very avant-garde, you are also invincible.',
 "No one can compare to me, it's too difficult.",
 'I want to ask which car around 2 million would be better. I am thinking of buying insurance recently, afraid that my own car will get into an accident.',
 "But he integrated everything that he couldn't compete with.",
 'The hardness of Luo Yonghao']

In [20]:
aito_comments['translation'] = aito_comments['comment'].map(translate_comment)

In [21]:
aito_comments.to_excel("Data/annotated_translated_aito_comment.xlsx")

In [18]:
li_comments['translation'] = li_comments['comment'].map(translate_comment)

In [19]:
li_comments.to_excel("Data/annotated_translated_li_comment.xlsx")